## Coding Example for Coordination Numbers
- It may be easier to understand this example if you run it cell-by-cell and not the whole way through, but this is just a recommendation!

In the cell below are all the programs you are going to need for the calculation. Go ahead and `import` them.

In [40]:
import nglview as ngl
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt

Now we are going to need to generate RDF calculations of the atoms that we are doing the coordination number calculation with. If you have not done a RDF before, I will show it down below. If you have, go ahead and copy your code into the same notebook you will be using to calculate coordination numbers. Skip to [here](#01)

A RDF (radial distribution function) is going to calculate the probability of seeing one atom around another atom which you can use to match with your coordination number results 
* high probability at a certain distance = high number of specific atom at that certain distance

## RDF Calculation
What you first want to do is read in your trajectories and give it a varaible. I used `traj`, but you can name it any way you want. Look below!

In [16]:
traj = md.load('/homes/kndiaye/code_materials/coordination_numbers/output.h5')

After loading the trajectories, we now want to make sure we know the element symbol of the atoms we are working with. Sometimes the PDB file will represent the stom with different symbols that aren't customed to the Periodic Table. Just to make sure that this isn't the case, we can examine what the PDB file says by looking through the trajectory. 

In my system, I wanted to check that Oxygen had the elememt symbol 'O'. To do so, we want to take a closer look at the molecule and it will tell us what atoms are in the molecule. There are tons of molecules in my system though, so looking through the PDB will help us condense what molecule we want the code to take a look at.

Navigate to the PDB file for your trajectory. So how you want to know the index number is by looking for the `TER` in the PDB file. That number indicates where the values and atoms of a molecule end and a new one begins. 

Take a look at my pdb file in the materials folder titled 'md_redo.pdb' for a clearer idea! The first `TER` has a number of 226 indicating I had 226 molecules of BMIM and the second `TER` has a number of 226 indicating that I had 226 molecules of BF4. This makes sense because that is what I had in my system using Packmol. Now the molecule I really wanna look at is H2O. So if I add 226 + 226, it should tell me the starting index of the last molecule in my system: H2O.

In the below cell is the code to get the that index and the atoms in it. And it does!
- **Note**
The 'VS' is just a drude particle that I had in my system

In [17]:
#examining water (should get H,H,O)
[i.element.symbol for i in
traj.topology.atoms if i.residue.index==452]

['O', 'H', 'H', 'VS', 'VS']

Now we can move onto getting the indices for the atoms. This is especially helpful for when you have the same atom type in different molecules and to get rid of drude particles if you have any. 

In my system, I just want the oxygen indices. Look below under 'getting indices'! I have created a list of all the oxygen atoms in the system and in my case, oxygen is the only atom of its kind in the entire system so I do not need to worry about anymore specification.

**Although** if I wanted to take a look at BMIM, which has hydrogens, and H2O, which also has hydrogens, I have to add more specification. Look under the comment that says 'more specifications'. The only extra thing you have to add is what molecule you want the hydrogen atoms to come from. In this case, I wrote it to look at the hydrogen atoms in water.

In [18]:
#getting indices
hoh_os = [i.index for i in traj.topology.atoms if i.element.symbol == 'O']

#more specification
hoh_hs = [i.index for i in traj.topology.atoms if 
          ((i.element.symbol == 'H') and (i.residue.name == 'HOH'))]

We now have everything we need for the RDF. Luckily, md.traj has a RDF calculation built in so all we need to do is add in what two atoms we want it to look at.

In my system, I wanted to get a RDF between only the oxygen atoms. Look below! For your pairs, you can put the same list of atoms twice (which is what I did). But if you wanted to evaluate a pair of two different atoms, write it as such.
- **Note** You can name your RDF as you want, I just found it easier to use 'gr' and then put the two atom symbols of what the RDF is calculating

In [19]:
#calculating the rdf betweeen oxygen atoms
grOO = md.compute_rdf(traj,
                          traj.topology.select_pairs(hoh_os, hoh_os), n_bins=100, r_range=[0.0, 2.0])

Now that we have calculated the RDF, we can move onto the main part: coordination numbers

## Coordination Numbers *still working on editing it <a class="anchor" id="01"></a>

Using the RDF(s) of the pair of atoms we want to calculate, we're going to check their bin width and define it. This is apart of the coordination number equation. 

I want to see the specific number of how many waters are around each other so I will use my RDF of oxygen-oxygen (grOO) only. Look below!

In [34]:
# Check bin width
grOO[0][1]- grOO[0][0]


0.02

In [35]:
# Define bin width 
bin_width = grOO[0][1] - grOO[0][0]

I mentioned in the README.txt file that you are going to want your output.dat file on hand and this is where we are going to need it. You need to calculate the edge length of the box and the number density using the values in there.

To get the edge length of the box, you need the volume of the box. You can find this in your output.dat file. You can check mine as reference for what it should look like. The size of my box didn't change through the MD simulation so I used '72.59'.
Then, I put it to the exponent of '1/3'. The volume of a cube is length cubed so to get the edge length which you want to take the volume to the '1/3' power.
Finally, multiple it '10' (this is just to get out of the nm^3 units into angstrons). The code is down below!

In [36]:
#length of box
72.59**(1/3)*10

41.715001685519255

To get the number density...
**i cannot access my notebook today so this step will be added in as soon as i get access to it**

Lastly, we are going to need the y values of the RDF so we can get this with code down below!

In [37]:
np.sum(grOO[1])

112.15811593523051

Now we just need to put it all together. The equation should be:


'4 * pi * number_density * np.sum(grXX[1] * grXX[O] **range_from_RDF * bin_width'.

I've written the code down below!

In [38]:
4*np.pi*0.413*np.sum(grOO[1]*grOO[0]**2*bin_width)

13.84168366782096

* *Here I will explain more about what this does when I get notebook access*

Now we want to calculate coordination numbers for the *entire* system (with the atoms you decided on) with this code down below

In [39]:
cn = np.zeros(100)

for i, n in enumerate(grOO[0]):
    cn[i] = 4*np.pi*0.413*np.sum(grOO[1]*grOO[0]**2*bin_width, where=(grOO[0] <= n))
cn

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.15593619e-03, 9.12310070e-02, 2.18207452e-01, 2.69277587e-01,
       2.90056421e-01, 3.03801500e-01, 3.16554848e-01, 3.29542748e-01,
       3.42978809e-01, 3.57241934e-01, 3.73028095e-01, 3.89792953e-01,
       4.08294447e-01, 4.28077848e-01, 4.48729624e-01, 4.71042318e-01,
       4.93982280e-01, 5.20264448e-01, 5.49895910e-01, 5.83510594e-01,
       6.21666534e-01, 6.65487319e-01, 7.14476605e-01, 7.69323459e-01,
       8.28684480e-01, 8.89589305e-01, 9.50707951e-01, 1.01264662e+00,
       1.07646695e+00, 1.14204478e+00, 1.21057893e+00, 1.28280711e+00,
       1.36025232e+00, 1.44254887e+00, 1.52642391e+00, 1.61398602e+00,
       1.70677171e+00, 1.80485035e+00, 1.90660237e+00, 2.01283380e+00,
       2.12340025e+00, 2.23925858e+00, 2.35846007e+00, 2.48501483e+00,
      

You can graph out your coordination numbers if you would like, but that concludes the code for being able to do so! 

**Slack me with any comments on how to improve this example, thanks**!